# Exercise 3: Fix a Puma script after a UI change
In this exercise, we will look at what happens when an app update breaks Puma code due to UI changes.

## 1. Run the script
Below is a puma script we want to execute.
It should do the following steps in Google Chrome:
- Go to google.com
- Bookmark the current page
- Go to wikipedia.org
- Open a new tab, and go to imdb.com
- Load the first saved bookmark
- Switch to the first open tab
- Open incognito mode and search for "DFRWS is awesome!"


In [ ]:
from puma.apps.android.google_chrome.google_chrome import GoogleChromeActions
# Set device udid. If your device udid is different, run adb devices and change the udid
device_udid = "YOUR DEVICE UDID"
chrome_actions = GoogleChromeActions(device_udid)

In [ ]:
chrome_actions.go_to("google.com")
chrome_actions.bookmark_page()
chrome_actions.go_to("en.wikipedia.org")
chrome_actions.go_to("www.imdb.com", new_tab=True)
chrome_actions.load_bookmark()
chrome_actions.switch_to_tab()
chrome_actions.go_to_incognito("DFRWS is awesome!")

## 2. Debug
As you might have noticed, the script exits with an exception. An app update seems to have changes the UI a bit,
and now our Puma code no logner works!
Changes to applications happen regularly, these changes can break your app automation script that uses Puma.Therefore Puma
needs to be maintained to maintain support for newer app versions when they are released. This exercise is about experiencing
an updated application which results in a broken app automation script.
What are the changes required to make the script useable again, and what could potentially be done to lower the risk of small changes resulting in failing scripts?
Below are all the steps of the script separately for easier debugging.

In the steps below three different (manufactured) problems occur, can you identify and fix them?  
**NOTE**: The steps themselfs are not the problem. The GoogleChromeActions class contains the bugs.  

**HINT**: Remember the XPath parameters, and make sure that the changes are backwards compatible.


In [ ]:
chrome_actions.go_to("google.com")

In [ ]:
chrome_actions.bookmark_page()

In [ ]:
chrome_actions.go_to("en.wikipedia.org")

In [ ]:
chrome_actions.go_to("www.imdb.com", new_tab=True)

In [ ]:
chrome_actions.load_bookmark()

In [ ]:
chrome_actions.switch_to_tab()

In [ ]:
chrome_actions.go_to_incognito("DFRWS is awesome!")

<details>
  <summary style="font-size: 24px;">Solution 1</summary>
  In the `go_to` method, we try to switch to a different tab. The content of the description of the button has changed and is no longer `Switch`. This needs to be changed to a XPath Contains query.
</details>

<details>
  <summary style="font-size: 24px;">Solution 2</summary>
  In the `bookmark_page` method, the content of the description has changed from lower-case to upper-case. A case-insensitive match should be used to keep it backwards compatible.
</details>

<details>
  <summary style="font-size: 24px;">Solution 3</summary>
  In the `go_to_incognito` method, we try to open a window by clicking the icon with three buttons. The content of the description of the button has changed and is no longer `Customize`. This needs to be changed to a XPath Contains query.
</details>